 # Opis notatnika
 Przygotowanie raportu używając `Dash` na podstawie danych z roku 2020.

 # Wymagania biznesowe
 Po prezentacji wyników analizy na niezależnym spotkaniu, zostały określone następujące obszary raportowania odsetka opóźnień lotów:  
 1. Wyświetlanie TOP 10 (w sensie najwyższego odsetka opóźnień) lotnisk na podstawie liczby odlotów, wyświetlona ma być również informacja o liczbie przylotów - widok `top_airports_by_departure`.  
 1. Wyświetlenie TOP 10 (w sensie najwyższego odsetka opóźnień) tras lotów. Przy czym istotna dla nas jest kolejność, przykładowo trasa (Warszawa, Paryż) jest inna niż (Paryż, Warszawa). Dodatkowym wymaganiem jest, aby minimalna liczba lotów odbytych na trasie wynosiła co najmniej 10000 przelotów - widok `top_reliability_roads`.  
 1. Porównanie roku 2019 vs. 2020, aby śledzić wpływ COVID na realizację lotów. Interesują nas podejścia:  
     - miesiąc do miesiąca, przykładowo odsetek opoźnień styczeń 2019 vs. odsetek opoźnień styczeń 2020, odsetek opoźnień luty 2019 vs. odsetek opoźnień luty 2020 itd. - widok `year_to_year_comparision`,  
     - dzień do dnia, przykładowo odsetek opoźnień wtorek 2019 vs. odsetek opoźnień wtorek 2020 - widok `day_to_day_comparision`.  
 1. Dzienny, czyli jak danego dnia, globalnie wyglądał wskaźnik opóźnień lotów samolotu, tj. odsetek opóźnień 01-01-2019, odsetek opóźnień 02-01-2019 itd.

 # Przygotowanie bazy danych
 Na bazie danych umieszczone są dane na schemacie `public`. Ponieważ zgodnie z wymaganiami otrzymuje nowy obszar wykorzystania danych, stworze schemat dedykowany - `reporting`.  
 Dalej stworzę widoki, które odpowiedzą na zadane wcześniej pytania.

 # Aktualizacja bazy danych
 Skonfiguruj połączenie do bazy danych.

In [1]:
import pandas as pd
import psycopg2
import matplotlib as plt
import matplotlib.pyplot as plt

 ## Konfiguracja połączenia

In [2]:
username = ''
password = ''

host = 'localhost'
database = 'airlines'
port = 5432

In [3]:
con = psycopg2.connect(user=username, password=password, host=host, database=database)
cursor = con.cursor()

 ## Wczytanie pliku `reporting.sql`

In [4]:
with open('../sql/reporting.sql', 'r') as schema_file:
    schema_queries = schema_file.read()

In [5]:
cursor.execute('CREATE SCHEMA IF NOT EXISTS reporting');

queries_list = schema_queries.split(';')

In [6]:
for queries in queries_list:
    print(queries)
    cursor.execute(queries)

CREATE SCHEMA IF NOT EXISTS reporting


/*

Tutaj napisz definicjÄ™ widoku reporting.flight, ktĂłra:
- bÄ™dzie usuwaÄ‡ dane o lotach anulowanych `cancelled = 0`
- bÄ™dzie zawieraÄ‡ kolumnÄ™ `is_delayed`, zgodnie z wczeĹ›niejszÄ… definicjÄ… tj. `is_delayed = 1 if dep_delay_new > 0 else 0` (zaimplementowana w SQL)
*/
CREATE OR REPLACE VIEW reporting.flight AS
SELECT
    *,
    CASE WHEN dep_delay_new > 0 THEN 1 ELSE 0 END AS is_delayed
FROM
    flight
WHERE
    cancelled = 0


/*
Tutaj napisz definicjÄ™ widoku reporting.top_reliability_roads, ktĂłra bÄ™dzie zawieraĹ‚a nastÄ™pujÄ…ce kolumny:
- `origin_airport_id`,
- `origin_airport_name`,
- `dest_airport_id`,
- `dest_airport_name`,
- `year`,
- `cnt` - jako liczba wykonananych lotĂłw na danej trasie,
- `reliability` - jako odsetek opĂłĹşnieĹ„ na danej trasie,
- `nb` - numerowane od 1, 2, 3 wedĹ‚ug kolumny `reliability`. W przypadku takich samych wartoĹ›ci powino zwrĂłciÄ‡ 1, 2, 2, 3... 
PamiÄ™taj o tym, ĹĽe w wyniku powinny pojawiÄ‡ siÄ™ t

In [7]:
con.commit()

In [11]:
con.close()

Wszystko wygląda OK :) Możesz przejść do kolejnego zadania.
